# Кодирование свечей по Лиховидову

Кодирование свечей по Лиховидову.  
Параметры большой, средний, маленький берутся по перцентилям предыдущих свечей за обозначенный период.  

In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
# Параметры по которым будут производиться расчеты величины теней и тела свечи
PERIOD = 30  # Период для подсчета значений квантилей.
QUAN_MIN = 0.25  # Нижняя граница ниже которой параметр считается маленьким.
QUAN_MAX = 0.75  # Веерхняя граница выше которой параметр считается большим.

In [3]:
# Загрузка данных для Colab с Гугл диска
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

connection = sqlite3.connect(r'/content/drive/MyDrive/data_quote_db/RTS_futures_day.db', check_same_thread=True)  # Создание соединения с БД

Mounted at /content/drive


In [4]:
# # Загрузка данных из БД для локальной БД
# connection = sqlite3.connect(fr'c:\Users\Alkor\gd\data_quote_db\RTS_futures_day.db', check_same_thread=True)  # Создание соединения с БД

In [5]:
with connection:
  df = pd.read_sql('SELECT TRADEDATE, OPEN, LOW, HIGH, CLOSE FROM Day', connection)  # Загрузка данных из БД

df.columns = map(str.lower, df.columns)  # Название колонок в нижний регистр
df

,tradedate,open,low,high,close
0,2015-01-05,76930.0,72470.0,78980.0,74600.0
1,2015-01-06,74470.0,71200.0,74610.0,73480.0
2,2015-01-08,73490.0,71000.0,81380.0,79980.0
3,2015-01-09,79950.0,74450.0,81050.0,77650.0
4,2015-01-12,77210.0,73180.0,77550.0,73900.0
...,...,...,...,...,...
2325,2024-04-15,115230.0,114530.0,115560.0,115310.0
2326,2024-04-16,115240.0,113620.0,115340.0,113690.0
2327,2024-04-17,113740.0,113190.0,114250.0,113780.0
2328,2024-04-18,113760.0,113060.0,114860.0,114700.0


In [6]:
# Подсчет размеров теней и тела свечи
df['size_hi'] = df.apply(lambda x: abs(x.high - x.open), axis=1)
df['size_body'] = df.apply(lambda x: abs(x.open - x.close), axis=1)
df['size_lo'] = df.apply(lambda x: abs(x.open - x.low), axis=1)

df

,tradedate,open,low,high,close,size_hi,size_body,size_lo
0,2015-01-05,76930.0,72470.0,78980.0,74600.0,2050.0,2330.0,4460.0
1,2015-01-06,74470.0,71200.0,74610.0,73480.0,140.0,990.0,3270.0
2,2015-01-08,73490.0,71000.0,81380.0,79980.0,7890.0,6490.0,2490.0
3,2015-01-09,79950.0,74450.0,81050.0,77650.0,1100.0,2300.0,5500.0
4,2015-01-12,77210.0,73180.0,77550.0,73900.0,340.0,3310.0,4030.0
...,...,...,...,...,...,...,...,...
2325,2024-04-15,115230.0,114530.0,115560.0,115310.0,330.0,80.0,700.0
2326,2024-04-16,115240.0,113620.0,115340.0,113690.0,100.0,1550.0,1620.0
2327,2024-04-17,113740.0,113190.0,114250.0,113780.0,510.0,40.0,550.0
2328,2024-04-18,113760.0,113060.0,114860.0,114700.0,1100.0,940.0,700.0


In [7]:
# Подсчет и запись в колонки заданных по условию квантилей для теней и тела свечи
df['q_hi_min'] = df.size_hi.rolling(window=PERIOD).quantile(QUAN_MIN)  # Минимальный заданный квантиль для верхней тени свечи
df['q_hi_max'] = df.size_hi.rolling(window=PERIOD).quantile(QUAN_MAX)  # Максимальный заданный квантиль для верхней тени свечи
df['q_body_min'] = df.size_body.rolling(window=PERIOD).quantile(QUAN_MIN)
df['q_body_max'] = df.size_body.rolling(window=PERIOD).quantile(QUAN_MAX)
df['q_lo_min'] = df.size_lo.rolling(window=PERIOD).quantile(QUAN_MIN)
df['q_lo_max'] = df.size_lo.rolling(window=PERIOD).quantile(QUAN_MAX)

df = df.dropna().reset_index(drop=True)
df

,tradedate,open,low,high,close,size_hi,size_body,size_lo,q_hi_min,q_hi_max,q_body_min,q_body_max,q_lo_min,q_lo_max
0,2015-02-16,90980.0,88920.0,92140.0,89320.0,1160.0,1660.0,2060.0,1160.0,4175.0,1150.0,2982.5,1077.5,2717.5
1,2015-02-17,89400.0,88060.0,93250.0,88410.0,3850.0,990.0,1340.0,1160.0,4175.0,1027.5,2982.5,1077.5,2500.0
2,2015-02-18,88440.0,88430.0,93610.0,92450.0,5170.0,4010.0,10.0,1242.5,4297.5,1150.0,3250.0,1055.0,2497.5
3,2015-02-19,92500.0,88280.0,93190.0,90420.0,690.0,2080.0,4220.0,1160.0,4175.0,1150.0,2982.5,1055.0,2500.0
4,2015-02-20,90430.0,89750.0,92630.0,90790.0,2200.0,360.0,680.0,1242.5,4175.0,1027.5,2982.5,810.0,2435.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2296,2024-04-15,115230.0,114530.0,115560.0,115310.0,330.0,80.0,700.0,292.5,905.0,265.0,867.5,212.5,890.0
2297,2024-04-16,115240.0,113620.0,115340.0,113690.0,100.0,1550.0,1620.0,265.0,885.0,265.0,867.5,235.0,937.5
2298,2024-04-17,113740.0,113190.0,114250.0,113780.0,510.0,40.0,550.0,265.0,862.5,185.0,867.5,235.0,937.5
2299,2024-04-18,113760.0,113060.0,114860.0,114700.0,1100.0,940.0,700.0,265.0,885.0,265.0,925.0,235.0,937.5


In [8]:
def candle_code(open, close, size_hi, size_body, size_lo, q_hi_min, q_hi_max, q_body_min, q_body_max, q_lo_min, q_lo_max) -> str:
    """
    Кодирование свечей по Лиховидову
    """
    code_str: str = ''  # Строка в которую будем собирать код свечи
    # Свеча на понижение (медвежья)
    if open > close:  # Свеча на понижение (медвежья)
        code_str += '0'
        # Для тела медвежьей свечи
        if size_body > q_body_max:  # 00 - медвежья свеча с телом больших размеров
            code_str += '00'
        elif size_body >= q_body_min:  # 01 - медвежья свеча с телом средних размеров
            code_str += '01'
        elif size_body > 0:  # 10 - медвежья свеча с телом небольших размеров
            code_str += '10'
        # Для верхней тени медвежьей свечи
        if size_hi > q_hi_max:  # 11 - верхняя тень больших размеров
            code_str += '11'
        elif size_hi >= q_hi_min:  # 10 - верхняя тень средних размеров
            code_str += '10'
        elif size_hi > 0:  # 01 - верхняя тень небольших размеров
            code_str += '01'
        else:  # 00 - верхняя тень отсутствует
            code_str += '00'
        # Для нижней тени медвежьей свечи
        if size_lo > q_lo_max:  # 00 - нижняя тень больших размеров
            code_str += '00'
        elif size_lo >= q_lo_min:  # 01 - нижняя тень средних размеров
            code_str += '01'
        elif size_lo > 0:  # 10 - нижняя тень небольших размеров
            code_str += '10'
        else:  # 11 - нижняя тень отсутствует
            code_str += '11'
    # Свеча на повышение (бычья)
    elif open < close:  # Свеча на повышение (бычья)
        code_str += '1'
        # Для тела бычьей свечи
        if size_body > q_body_max:  # 11 - бычья свеча с телом больших размеров.
            code_str += '11'
        elif size_body >= q_body_min:  # 10 - бычья свеча с телом средних размеров
            code_str += '10'
        elif size_body > 0:  # 01 - бычья свеча с телом небольших размеров
            code_str += '01'
        # Для верхней тени бычьей свечи
        if size_hi > q_hi_max:  # 11 - верхняя тень больших размеров
            code_str += '11'
        elif size_hi >= q_body_min:  # 10 - верхняя тень средних размеров
            code_str += '10'
        elif size_hi > 0:  # 01 - верхняя тень небольших размеров
            code_str += '01'
        else:  # 00 - верхняя тень отсутствует
            code_str += '00'
        # Для нижней тени бычьей свечи
        if size_lo > q_lo_max:  # 00 - нижняя тень больших размеров
            code_str += '00'
        elif size_lo >= q_lo_min:  # 01 - нижняя тень средних размеров
            code_str += '01'
        elif size_lo > 0:  # 10 - нижняя тень небольших размеров
            code_str += '10'
        else:  # 11 - нижняя тень отсутствует
            code_str += '11'
    # Дожи
    else:  # Дожи
        if size_hi > size_lo:  # Верхняя тень больше, медвежий дожи
            code_str += '011'
        else:  # Верхняя тень меньше, бычий дожи
            code_str += '100'
        # Для верхней тени дожи
        if size_hi > q_hi_max:  # 11 - верхняя тень больших размеров
            code_str += '11'
        elif size_hi >= q_hi_min:  # 10 - верхняя тень средних размеров
            code_str += '10'
        elif size_hi > 0:  # 01 - верхняя тень небольших размеров
            code_str += '01'
        else:  # 00 - верхняя тень отсутствует
            code_str += '00'
        # Для нижней тени дожи
        if size_lo > q_hi_max:  # 00 - нижняя тень больших размеров
            code_str += '00'
        elif size_lo >= q_lo_min:  # 01 - нижняя тень средних размеров
            code_str += '01'
        elif size_lo > 0:  # 10 - нижняя тень небольших размеров
            code_str += '10'
        else:  # 11 - нижняя тень отсутствует
            code_str += '11'
    return code_str


In [9]:
# Добавление колонки кода свечи
df['candle_code'] = df.apply(lambda x: candle_code(
    x.open,
    x.close,
    x.size_hi,
    x.size_body,
    x.size_lo,
    x.q_hi_min,
    x.q_hi_max,
    x.q_body_min,
    x.q_body_max,
    x.q_lo_min,
    x.q_lo_max
    ), axis=1)  # Заполняем столбец candle_code

df['up/down'] = df[['open', 'close']].apply(lambda x: 1 if (x.close > x.open) else 0, axis=1)  # Добавление колонки напрвления свечи
# df['prev_cc'] = df.candle_code.shift(1)  # Добавление колонки кода предыдущей свечи
df['prev_cc'] = df.candle_code.shift(-1)  # Добавление колонки кода предыдущей свечи
df = df[['tradedate', 'open', 'low', 'high', 'close', 'up/down', 'candle_code', 'prev_cc']]
df = df.dropna().reset_index(drop=True)
df

,tradedate,open,low,high,close,up/down,candle_code,prev_cc
0,2015-02-17,89400.0,88060.0,93250.0,88410.0,0,0101001,0011001
1,2015-02-18,88440.0,88430.0,93610.0,92450.0,1,1111110,0101001
2,2015-02-19,92500.0,88280.0,93190.0,90420.0,0,0010100,1111110
3,2015-02-20,90430.0,89750.0,92630.0,90790.0,1,1011010,0010100
4,2015-02-24,90730.0,86020.0,91760.0,88650.0,0,0010100,1011010
...,...,...,...,...,...,...,...,...
2295,2024-04-15,115230.0,114530.0,115560.0,115310.0,1,1011001,1111101
2296,2024-04-16,115240.0,113620.0,115340.0,113690.0,0,0000100,1011001
2297,2024-04-17,113740.0,113190.0,114250.0,113780.0,1,1011001,0000100
2298,2024-04-18,113760.0,113060.0,114860.0,114700.0,1,1111101,1011001


In [10]:
# Проверка столбца кода свечи на количество знаков
# df['size_str_cc'] = df.apply(lambda x: len(x.prev_cc), axis=1)  # Размер строки кода свечи
# df.size_str_cc.unique()

In [11]:
# Создание фичей из candle code предыдущей свечи
for i in range(7):
    df[f'f_{i}'] = df.apply(lambda x: int(x.prev_cc[i]), axis=1)

df

,tradedate,open,low,high,close,up/down,candle_code,prev_cc,f_0,f_1,f_2,f_3,f_4,f_5,f_6
0,2015-02-17,89400.0,88060.0,93250.0,88410.0,0,0101001,0011001,0,0,1,1,0,0,1
1,2015-02-18,88440.0,88430.0,93610.0,92450.0,1,1111110,0101001,0,1,0,1,0,0,1
2,2015-02-19,92500.0,88280.0,93190.0,90420.0,0,0010100,1111110,1,1,1,1,1,1,0
3,2015-02-20,90430.0,89750.0,92630.0,90790.0,1,1011010,0010100,0,0,1,0,1,0,0
4,2015-02-24,90730.0,86020.0,91760.0,88650.0,0,0010100,1011010,1,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2295,2024-04-15,115230.0,114530.0,115560.0,115310.0,1,1011001,1111101,1,1,1,1,1,0,1
2296,2024-04-16,115240.0,113620.0,115340.0,113690.0,0,0000100,1011001,1,0,1,1,0,0,1
2297,2024-04-17,113740.0,113190.0,114250.0,113780.0,1,1011001,0000100,0,0,0,0,1,0,0
2298,2024-04-18,113760.0,113060.0,114860.0,114700.0,1,1111101,1011001,1,0,1,1,0,0,1


# Прогнозирование используя код свечи по Лиховидову и нейронные сети

Создание dataset

In [12]:
# dataset = df[['tradedate', 'f_0', 'f_1', 'f_2', 'f_3', 'f_4', 'f_5', 'f_6', 'up/down']].values
dataset = df[['f_0', 'f_1', 'f_2', 'f_3', 'f_4', 'f_5', 'f_6', 'up/down']].values
dataset = dataset.astype(np.float32)  # Смена типа для корректой работы Keras
dataset

array([[0., 0., 1., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 1.],
       [1., 1., 1., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 1., ..., 0., 1., 1.],
       [1., 1., 1., ..., 0., 1., 1.]], dtype=float32)

In [13]:
X = dataset[:,0:7]  # Все, что стоит перед запятой, относится к строкам массива, а все, что стоит после запятой, относится к столбцам массивов.
X

array([[0., 0., 1., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       [1., 1., 1., ..., 1., 1., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 1., ..., 0., 0., 1.],
       [1., 1., 1., ..., 1., 0., 1.]], dtype=float32)

In [14]:
Y = dataset[:,7]
Y

array([0., 1., 0., ..., 1., 1., 1.], dtype=float32)

Используем функцию, называемую min-max scaler, которая масштабирует набор данных так, чтобы все входные характеристики находились в диапазоне от 0 до 1 включительно:

In [15]:
# Данные уже нормализованы, выполнение этой ячейки не нужно.
#from sklearn import preprocessing
#min_max_scaler = preprocessing.MinMaxScaler()
#X_scale = min_max_scaler.fit_transform(X)
#X_scale

Теперь мы перешли к последнему этапу обработки данных, который заключается в разделении нашего набора данных на обучающий набор, набор проверки и набор тестов.  
Мы будем использовать код из scikit-learn под названием "train_test_split", который, как следует из названия, разбивает наш набор данных на обучающий и тестовый наборы. Сначала мы импортируем необходимые библиотеки:

In [16]:
from sklearn.model_selection import train_test_split

Затем разделите свой набор данных следующим образом:  
Это говорит scikit-learn, что ваш размер val_and_test будет составлять 30% от общего набора данных. Код сохранит разделенные данные в первые четыре переменные слева от знака равенства, как следует из имен переменных.

In [17]:
X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(X, Y, test_size=0.3)

К сожалению, эта функция только помогает нам разделить наш набор данных на две части. Поскольку нам нужен отдельный набор проверки и тестовый набор, мы можем использовать ту же функцию, чтобы снова выполнить разделение для val_and_test:  
Приведенный выше код разделит размер val_and_test поровну на набор проверки и набор тестов.

Итак, теперь у нас есть в общей сложности шесть переменных для наших наборов данных, которые мы будем использовать:

- X_train (10 входных функций, 70% от полного набора данных)
- X_val (10 входных функций, 15% от полного набора данных)
- X_test (10 входных параметров, 15% от полного набора данных)
- Y_train (1 метка, 70% от полного набора данных)
- Y_val (1 метка, 15% от полного набора данных)
- Y_test (1 метка, 15% от полного набора данных)  
Если вы хотите увидеть, какие формы имеют массивы для каждого из них (т. Е. Какие они имеют размеры), Просто запустите

In [18]:
# X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5)
# print(X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape)

In [19]:
# import keras
# print(keras.__version__)

In [20]:
from keras.models import Sequential
from keras.layers import Dense

Затем мы указываем это в нашей последовательной модели Keras следующим образом:

In [21]:
# Создание модели нейронной сети
model = Sequential([
    Dense(64, activation='relu', input_shape=(7,)),
    Dense(64, activation='relu'),
    # Dense(1, activation='sigmoid'),
    Dense(2, activation='softmax'),
])

In [22]:
# Компиляция модели
# model.compile(optimizer='sgd',
#               loss='binary_crossentropy',
#               metrics=['accuracy'])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Обучение модели на данных
# hist = model.fit(X_train, Y_train,
#           batch_size=32, epochs=100,
#           validation_data=(X_val, Y_val))

model.fit(X_train, Y_train,
          batch_size=32, epochs=10000,
          validation_data=(X_val_and_test, Y_val_and_test))

Streaming output truncated to the last 5000 lines.
Epoch 7501/10000
51/51 [==============================] - 0s 4ms/step - loss: 0.6780 - accuracy: 0.5540 - val_loss: 0.7380 - val_accuracy: 0.5101
Epoch 7502/10000
51/51 [==============================] - 0s 4ms/step - loss: 0.6778 - accuracy: 0.5559 - val_loss: 0.7381 - val_accuracy: 0.5087
Epoch 7503/10000
51/51 [==============================] - 0s 4ms/step - loss: 0.6779 - accuracy: 0.5559 - val_loss: 0.7378 - val_accuracy: 0.5087
Epoch 7504/10000
51/51 [==============================] - 0s 4ms/step - loss: 0.6780 - accuracy: 0.5578 - val_loss: 0.7371 - val_accuracy: 0.4971
Epoch 7505/10000
51/51 [==============================] - 0s 4ms/step - loss: 0.6780 - accuracy: 0.5547 - val_loss: 0.7372 - val_accuracy: 0.5130
Epoch 7506/10000
51/51 [==============================] - 0s 4ms/step - loss: 0.6781 - accuracy: 0.5565 - val_loss: 0.7374 - val_accuracy: 0.5130
Epoch 7507/10000
51/51 [==============================] - 0s 4ms/step - l

In [ ]:
# Оценка модели на тестовых данных
test_loss, test_acc = model.evaluate(X_val_and_test, Y_val_and_test)

print('Test accuracy:', test_acc)

22/22 [==============================] - 0s 2ms/step - loss: 0.7577 - accuracy: 0.5014
Test accuracy: 0.5014492869377136
